In [1]:
! pip install datasets transformers[sentencepiece] sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requ

In [21]:
from huggingface_hub import notebook_login
notebook_login()

# Fine-tuning a model on a translation task
we will use the `WMT dataset`. Dataset composed from collection of sources, including news commentaries and parlimant proceedings

In [3]:
model_checkpoint='Helsinki-NLP/opus-mt-en-ro'
# Helsinki-NLP/opus-mt-en-ro --> Developed by University of Helsinki
# en-ro --> The model name indicates that it is trained to translate from English to Roman
# OPUS --> Open Parallel Corpus is a growing collection of translated texts from web. It collects these parallel texts and preprocess them to create training data for machine translation systems

#Loading the dataset
We will use `Dataset` library to download the data.

In [4]:
from datasets import load_dataset
import evaluate
raw_datasets=load_dataset('wmt16','ro-en')
metric=evaluate.load('sacrebleu')
# Sacreblue -->SecreBLUE aims to quantify how similar a machine-generated translation is to one or more high-quality human produced translation.
"""
Key Concepts -
  1. N-gram Matching-
      SacreBLUE counts how many n-gram in the machine generated translation also
      appear in the reference translation
  2. Precision Calculation-
      n-gram precision=(number of matching n-gram)/(total number of n-gram in machine )
  3. Clipping-
      To clip down multiple referenced translation, which might be inflate teh maximum occurance in the single reference
  4. Brevity Penalty-
      If machine translation is shorter than shorets reference translation , the overall BLUE score will peanalize
"""

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/362k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/342k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

'\nKey Concepts -\n  1. N-gram Matching-\n      SacreBLUE counts how many n-gram in the machine generated translation also\n      appear in the reference translation\n  2. Precision Calculation-\n      n-gram precision=(number of matching n-gram)/(total number of n-gram in machine )\n  3. Clipping-\n      To clip down multiple referenced translation, which might be inflate teh maximum occurance in the single reference\n  4. Brevity Penalty-\n      If machine translation is shorter than shorets reference translation , the overall BLUE score will peanalize\n'

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

In [6]:
raw_datasets['train'][0]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'A Strategic Development unit, to be established in the public sector, will direct the development in different sectors.', 'ro': 'O unitate de Dezvoltare Strategică, care va fi înfiinţată în sectorul public, va dirija dezvoltarea din diferitele sectoare.'}"
1,"{'en': '""This year, we will construct a part of the infrastructure and housing facilities, and we will install a ski lift for promotional usage of the ski tracks by the end of the year,"" says project manager Angel Nakov.', 'ro': '""Anul acesta vom construi o parte din infrastructură şi din centrele de cazare. De asemenea vom instala un telescaun pentru utilizarea promoţională a pârtiilor de schi până la sfârşitul anului"", declară managerul de proiect Angel Nakov.'}"
2,"{'en': 'For the first time, the census included a category of people --1.2% of the population or 81,000 -- who have never set foot in school.', 'ro': 'Recensământul a inclus pentru prima dată categoria persoanelor care nu au pus niciodată piciorul în şcoală, 1,2% din populaţie, respectiv 81 000 de cetăţeni.'}"
3,"{'en': 'in writing. - Energy efficiency (EE) is the most cost-efficient measure to reduce greenhouse gas emissions.', 'ro': 'în scris. - Eficiența energetică (EE) este cea mai rentabilă măsură pentru reducerea emisiilor de gaze cu efect de seră.'}"
4,"{'en': 'In Skopje on Friday, Nimetz met with Gruevski and Foreign Minister Antonio Milososki.', 'ro': 'Vineri, Nimetz s-a întâlnit la Skopie cu Gruevski şi cu ministrul de externe Antonio Milososki.'}"


In [9]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

# Preprocessing the data

In [10]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/817k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"
# We are setting source and target language, So the texts are processed properly

We are using `as_target_tokenizer` context manager. This will make sure the tokenzer uses the special token corresponding to the targets

In [12]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[10334, 1204, 3, 15, 8915, 27, 452, 59, 29579, 581, 23, 0], [235, 1705, 11, 32, 8, 1205, 5305, 59, 29579, 581, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [13]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [14]:
""" Primary role of this function is to take raw data and transform it into a format that a
 transformer-based trranslation model can understand and porcess it"""

max_input_length=128
max_target_length=128
source_lang='en'
target_lang='ro'

def preprocess_function(examples):
  # Extract source language text from input.
  inputs= [prefix + ex[source_lang] for ex in examples['translation']]
  targets= [ex[target_lang] for ex in examples['translation']]

  # Tokenize the input
  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)

    model_inputs['labels']=labels['input_ids']
    return model_inputs

In [15]:
# Apply the function on all pair of sentences in our datset.
tokenized_datasets=raw_datasets.map(preprocess_function,batched=True)

Map:   0%|          | 0/610320 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

# Fine-tuning the model

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model=AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Before Train our model need to provide the `Seq2SeqTrainingArguments`

In [17]:
batch_size=16
model_name=model_checkpoint.split('/')[-1]
args=Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True
)

In [18]:
# We need data collator -> which will pad input and label it
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [19]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-22-4b7ecf5c84dd>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ombhandwalkar38126 (ombhandwalkar38126-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.741300,1.288601,28.165000,37.496700


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59542]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=38145, training_loss=0.7720830595591759, metrics={'train_runtime': 6903.083, 'train_samples_per_second': 88.413, 'train_steps_per_second': 5.526, 'total_flos': 1.0082691528523776e+16, 'train_loss': 0.7720830595591759, 'epoch': 1.0})

In [24]:
trainer.push_to_hub()

events.out.tfevents.1746458787.55a4b1bee273.375.0:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/opus-mt-en-ro-finetuned-en-to-ro/commit/6da8e1050b75b892defc58cf26af1f774d57a762', commit_message='End of training', commit_description='', oid='6da8e1050b75b892defc58cf26af1f774d57a762', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/opus-mt-en-ro-finetuned-en-to-ro', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/opus-mt-en-ro-finetuned-en-to-ro'), pr_revision=None, pr_num=None)

In [26]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("OmBhandwalkar/opus-mt-en-ro-finetuned-en-to-ro")

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/299M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]